<font size="6">Lending Club Analysis</font>

Objective:
- Tasked by an Investment firm to analyze the possibility of investing in Lending Club loans as part of our portfolio. 


Tasks:
-	Perform EDA to understand dataset.
-	Impute missing data.
-	Select features, removing ones that could lead to data leakage or are unimportant.
-	Train models to predict loan defaults.
-	Calculate the best CAGR.
-	Compare with other investment assets.
-	Conclusions.
-	Future work.

## Load Packages and Data

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import seaborn as sns

plt.style.use("fivethirtyeight")
sns.set_style('whitegrid')
%matplotlib inline

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 151)
pd.set_option('display.max_rows', 50)

In [2]:
#data = pd.read_csv("G:\\Lending Club Data Files\\accepted_2007_to_2018Q4.csv")
data = pd.read_csv("data\accepted_2007_to_2018Q4.csv")

OSError: [Errno 22] Invalid argument: 'data\x07ccepted_2007_to_2018Q4.csv'

In [ ]:
data.sample(2)

## Understanding the Data

In [ ]:
data.shape

In [ ]:
data.tot_hi_cred_lim.isna().sum()

In [ ]:
data.loan_status.value_counts()

In [ ]:
for e, column in enumerate(data.columns):
    print (e, column)

Using the dictionary with the descriptions I will review the features available to determine which were obtained after the loan origination to avoid data leakage. We should only use information available at loan origination in order to avoid having a model that overestimates its predictive power. The following are features I found to be available at origination and that are relevant:

- id, loan_amnt, term, int_rate, grade, sub_grade, emp_title, home_ownership, annual_inc, verification_status, issue_d, loan_status (target), purpose, zip_code, addr_state, dti, earliest_cr_line, fico_range_low, fico_range_high, open_acc, pub_rec, initial_list_status, application_type, annual_inc_joint, dti_joint, verification_status_joint, pub_rec_bankruptcies

Column index: 0, 2, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, 20, 22, 23, 24, 26, 27, 28, 32, 33, 37, 56, 57, 58, 59, 109

In [ ]:
df = data.iloc[:,[0,2,5,6,8,9,11,12,13,14,15,16,20,22,23,24,26,27,28,32,33,37,56,57,58,59,109]]

Our analysis should only include observations where the target variable, loan_status, has a definite status in order to have a target that can be used to train our models. In this case, either the loan_status is "Fully Paid" or "Charged Off", which means it defaulted.

In [ ]:
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

### Exploring the Target Variable

In [ ]:
df['loan_status'].value_counts()

In [ ]:
plt.rcParams.update({'font.size': 14})
f, ax = plt.subplots(dpi=80)
langs = ['Fully Paid', 'Charged Off']
ax.pie(df['loan_status'].value_counts(), labels = langs, autopct='%1.1f%%', colors=('darkred','orangered'))
plt.show()
f.savefig('./images/pie_loan_status.png', facecolor=fig.get_facecolor(), transparent=True, dpi=500)